<a href="https://colab.research.google.com/github/aryanvakharia/MP3.2_private/blob/main/mp3_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import torch
import random
import numpy as np

seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
print(f"Random seed set as {seed}")

torch.cuda.empty_cache()

Random seed set as 42


In [ ]:
MAIN_DIR = "drive/MyDrive/cs410_mp3"

In [ ]:
!pip install transformers

In [ ]:
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import json
import pandas as pd

In [ ]:
df = pd.read_csv(f"{MAIN_DIR}/train_data.csv")
df.head()

,document,label
0,"Economy of India From Wikipedia, the free ency...",0
1,"Silicon From Wikipedia, the free encyclopedia ...",0
2,Call Us 1 - 603 - 244 - 6292 Follow Us 1 - 603...,1
3,Skip to main content .us Hello Select your add...,0
4,"Mucus From Wikipedia, the free encyclopedia Ju...",0


In [ ]:
train_texts, train_labels = df['document'].values, df['label'].values

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenized_texts = []
from tqdm import tqdm

max_seq_length = 512  # Maximum sequence length for BERT

for text in tqdm(train_texts):
    tokenized_texts.append(tokenizer(text, truncation=True, padding='max_length', max_length=max_seq_length, return_tensors='pt'))

100%|██████████| 1500/1500 [02:44<00:00,  9.11it/s]


In [ ]:
# Tokenize the texts and convert them to tensors
from sklearn.metrics import accuracy_score, f1_score, classification_report

input_ids = torch.cat([t['input_ids'] for t in tokenized_texts], dim=0)
attention_mask = torch.cat([t['attention_mask'] for t in tokenized_texts], dim=0)
labels = torch.tensor(train_labels)

# Create a dataset and data loader
dataset = TensorDataset(input_ids, attention_mask, labels)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

batch_size = 16
lr = 1e-5

train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

optimizer = AdamW(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# len(val_dataset)

# val_dataset[0]

In [ ]:
def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    for batch in tqdm(dataloader):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    return total_loss / len(dataloader)

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)
            correct_predictions += torch.sum(predictions == labels).item()
            print(labels.cpu())
            total_samples += labels.size(0)


    print(classification_report(predictions.cpu().numpy(), labels.cpu().numpy()))
    return total_loss / len(dataloader), correct_predictions / total_samples

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
num_epochs = 1
for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)

    print(f"Epoch {epoch + 1}/{num_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Val Accuracy: {val_accuracy:.2%}")
    model.save_pretrained(f"{MAIN_DIR}/fine_tuned_bert_epoch_{epoch+1}_lr_{lr}")

# Save the fine-tuned model
model.save_pretrained(f"{MAIN_DIR}/fine_tuned_bert")

  5%|▌         | 1/19 [00:00<00:10,  1.76it/s]

tensor([0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0], device='cuda:0')


 11%|█         | 2/19 [00:01<00:09,  1.75it/s]

tensor([1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1], device='cuda:0')


 16%|█▌        | 3/19 [00:01<00:09,  1.76it/s]

...

 95%|█████████▍| 18/19 [00:10<00:00,  1.77it/s]

tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')


100%|██████████| 19/19 [00:10<00:00,  1.78it/s]

tensor([0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1], device='cuda:0')
              precision    recall  f1-score   support

           0       1.00      0.80      0.89        10
           1       0.50      1.00      0.67         2

    accuracy                           0.83        12
   macro avg       0.75      0.90      0.78        12
weighted avg       0.92      0.83      0.85        12

Epoch 1/1 - Train Loss: 0.5096 - Val Loss: 0.4078 - Val Accuracy: 82.33%


KeyboardInterrupt: ignored

In [ ]:
# np.count_nonzero(labels.cpu().numpy() == 0)

c = 0
for i in labels.cpu().numpy():
  if i == 0:
    c += 1

print(c)
len(labels)

1139


1500

## Inference

In [ ]:
df_test = pd.read_csv(f"{MAIN_DIR}/test_data.csv")
df_test.head()

In [ ]:
texts_test = df_test['document'].values

In [ ]:
# To use the fine-tuned model for inference:
loaded_model = BertForSequenceClassification.from_pretrained(f"{MAIN_DIR}/fine_tuned_bert")
loaded_model.to(device)

In [ ]:
loaded_model.eval()

all_preds = []

with torch.no_grad():
  for text in tqdm(texts_test):
      tokenized_sentence = tokenizer(text, truncation=True, padding='max_length', max_length=max_seq_length, return_tensors='pt')
      input_ids = tokenized_sentence["input_ids"].to(device)
      attention_mask = tokenized_sentence["attention_mask"].to(device)

      outputs = loaded_model(input_ids, attention_mask=attention_mask)
      logits = outputs.logits
      predictions = torch.argmax(logits, dim=1)
      all_preds.extend(predictions.cpu().numpy())

In [ ]:
import csv
with open(f'{MAIN_DIR}/metamia_results.csv', mode='w') as csv_file: # for mp3.1, use filename 'mp3.1_results.csv'
    writer = csv.writer(csv_file)
    writer.writerow(['label'])
    for item in all_preds:
        writer.writerow([item])